In [1]:
import cv2
import colorsys
import numpy as np
import os
import sys

from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from keras.utils import multi_gpu_model
from PIL import Image, ImageFont, ImageDraw
from yolo3.model import yolo_body, yolo_eval, tiny_yolo_body
from yolo3.utils import letterbox_image
from yolo import YOLO

Using TensorFlow backend.


In [2]:
x1, y1, x2, y2 = -1, -1, -1, -1
path = "../1_02.avi"

In [3]:
def draw_rec(event, x, y, flags, param):
    global x1, y1, x2, y2, drawing, mode
    
    if event == cv2.EVENT_LBUTTONDOWN:
        x1, y1 = x, y
    if event == cv2.EVENT_LBUTTONUP:
        x2, y2 = x, y
        cv2.rectangle(param, (x1, y1), (x2, y2), (0, 0, 255), 3)

In [ ]:
cap = cv2.VideoCapture(path)
# while cap.isOpened():
ret, frame = cap.read()
# cv2.setMouseCallback('draw_rectangle', draw_rec, frame)

yolo = YOLO()

# SelectROI (bbox) (top, left, height, width)
bbox_area = cv2.selectROI('track area', frame)
print("Type(bbox_area): {} and value of bbox_area: {}".format(type(bbox_area), bbox_area))
while True:
    ok, frame = cap.read()
    
    image = Image.fromarray(frame)
    
    image, center, box_detect, obj_type = yolo.detect_image(image, bbox_area)
    frame = np.asarray(image)
    
    
    cv2.rectangle(frame, (bbox_area[0], bbox_area[1]), (bbox_area[0] + bbox_area[2], bbox_area[1] + bbox_area[3]), (255, 0, 0), 3)
    cv2.imshow("frame", frame)
    
    k = cv2.waitKey(1) & 0xff
    if k == ord('n'):
        continue
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()

../weights/ep069-loss46.542-val_loss45.218.h5 model, anchors, and classes loaded.
Type(bbox_area): <class 'tuple'> and value of bbox_area: (418, 215, 731, 689)
(416, 416, 3)
Found 0 boxes for img
1.9833417740001096
(416, 416, 3)
Found 0 boxes for img
0.08938275700029408
(416, 416, 3)
Found 0 boxes for img
0.0853224269999373
(416, 416, 3)
Found 0 boxes for img
0.08803436599964698
(416, 416, 3)
Found 0 boxes for img
0.08904287299992575
(416, 416, 3)
Found 0 boxes for img
0.08822384199993394
(416, 416, 3)
Found 0 boxes for img
0.08686631499995201
(416, 416, 3)
Found 0 boxes for img
0.08908291700026894
(416, 416, 3)
Found 0 boxes for img
0.09574705900013214
(416, 416, 3)
Found 0 boxes for img
0.0841022160002467
(416, 416, 3)
Found 0 boxes for img
0.10025494899991827
(416, 416, 3)
Found 0 boxes for img
0.08378785600007177
(416, 416, 3)
Found 0 boxes for img
0.0869273449998218
(416, 416, 3)
Found 0 boxes for img
0.09078582899974208
(416, 416, 3)
Found 0 boxes for img
0.09277394200034905
(416

(416, 416, 3)
Found 0 boxes for img
0.09706003400015106
(416, 416, 3)
Found 0 boxes for img
0.0874858279998989
(416, 416, 3)
Found 0 boxes for img
0.09475767200001428
(416, 416, 3)
Found 0 boxes for img
0.0896343340000385
(416, 416, 3)
Found 0 boxes for img
0.09612155300010272
(416, 416, 3)
Found 0 boxes for img
0.10743717700006528
(416, 416, 3)
Found 0 boxes for img
0.09163420400000177
(416, 416, 3)
Found 0 boxes for img
0.09377514000016163
(416, 416, 3)
Found 0 boxes for img
0.0931455250001818
(416, 416, 3)
Found 0 boxes for img
0.09509786200032977
(416, 416, 3)
Found 0 boxes for img
0.08884758800013515
(416, 416, 3)
Found 0 boxes for img
0.09743768699991051
(416, 416, 3)
Found 0 boxes for img
0.0975528909998502
(416, 416, 3)
Found 1 boxes for img
face 0.69 (563, 224) (592, 258)
0.08748812900012126
(416, 416, 3)
Found 1 boxes for img
face 0.36 (589, 230) (618, 262)
0.0961064019998048
(416, 416, 3)
Found 1 boxes for img
face 0.37 (607, 238) (638, 272)
0.09802917999968486
(416, 416, 3)

Found 0 boxes for img
0.0918804459997773
(416, 416, 3)
Found 0 boxes for img
0.08818446900022536
(416, 416, 3)
Found 0 boxes for img
0.09849289900012081
(416, 416, 3)
Found 0 boxes for img
0.09057827700007692
(416, 416, 3)
Found 0 boxes for img
0.09067406599979222
(416, 416, 3)
Found 0 boxes for img
0.09590183899990734
(416, 416, 3)
Found 0 boxes for img
0.08527983300018604
(416, 416, 3)
Found 0 boxes for img
0.08802427000000534
(416, 416, 3)
Found 0 boxes for img
0.1020143390001067
(416, 416, 3)
Found 0 boxes for img
0.08882139199977246
(416, 416, 3)
Found 0 boxes for img
0.10187720999965677
(416, 416, 3)
Found 0 boxes for img
0.0895810239999264
(416, 416, 3)
Found 0 boxes for img
0.09608852200017282
(416, 416, 3)
Found 0 boxes for img
0.09040119600012986
(416, 416, 3)
Found 0 boxes for img
0.08548724199999924
(416, 416, 3)
Found 0 boxes for img
0.09453789000008328
(416, 416, 3)
Found 0 boxes for img
0.0929848599998877
(416, 416, 3)
Found 0 boxes for img
0.08988440400025866
(416, 416,

(416, 416, 3)
Found 0 boxes for img
0.08616634100008014
(416, 416, 3)
Found 0 boxes for img
0.09716681899999458
(416, 416, 3)
Found 0 boxes for img
0.08878507500003252
(416, 416, 3)
Found 0 boxes for img
0.09175251600026968
(416, 416, 3)
Found 0 boxes for img
0.09564688599994042
(416, 416, 3)
Found 0 boxes for img
0.09021248899989587
(416, 416, 3)
Found 0 boxes for img
0.09097217499993349
(416, 416, 3)
Found 0 boxes for img
0.08621546300037153
(416, 416, 3)
Found 0 boxes for img
0.09146178199989663
(416, 416, 3)
Found 0 boxes for img
0.08782928400023593
(416, 416, 3)
Found 0 boxes for img
0.09309181799972066
(416, 416, 3)
Found 0 boxes for img
0.09692022399985945
(416, 416, 3)
Found 0 boxes for img
0.09144207700001061
(416, 416, 3)
Found 0 boxes for img
0.0926227929999186
(416, 416, 3)
Found 0 boxes for img
0.0904710290001276
(416, 416, 3)
Found 0 boxes for img
0.09676884199961933
(416, 416, 3)
Found 0 boxes for img
0.094290058000297
(416, 416, 3)
Found 0 boxes for img
0.09673584400024

Found 0 boxes for img
0.09413462300017272
(416, 416, 3)
Found 0 boxes for img
0.08664290900014748
(416, 416, 3)
Found 0 boxes for img
0.09597560300016994
(416, 416, 3)
Found 0 boxes for img
0.09408039299978554
(416, 416, 3)
Found 0 boxes for img
0.10218061299974579
(416, 416, 3)
Found 0 boxes for img
0.09215048399983061
(416, 416, 3)
Found 0 boxes for img
0.09830133200011915
(416, 416, 3)
Found 0 boxes for img
0.08602734399983092
(416, 416, 3)
Found 0 boxes for img
0.09058207500038407
(416, 416, 3)
Found 0 boxes for img
0.08730873000013162
(416, 416, 3)
Found 0 boxes for img
0.09164773000020432
(416, 416, 3)
Found 0 boxes for img
0.09104406000005838
(416, 416, 3)
Found 0 boxes for img
0.08572301899994272
(416, 416, 3)
Found 0 boxes for img
0.09491602199977933
(416, 416, 3)
Found 0 boxes for img
0.09430471899986514
(416, 416, 3)
Found 0 boxes for img
0.08810252399962337
(416, 416, 3)
Found 0 boxes for img
0.09651209900039248
(416, 416, 3)
Found 0 boxes for img
0.0848744939999051
(416, 4

...

(416, 416, 3)
Found 0 boxes for img
0.09259174599992548
(416, 416, 3)
Found 1 boxes for img
face 0.95 (700, 250) (731, 286)
0.08868181600018943
(416, 416, 3)
Found 1 boxes for img
face 0.89 (719, 249) (755, 285)
0.0918794560002425
(416, 416, 3)
Found 0 boxes for img
0.10102563800001008
(416, 416, 3)
Found 1 boxes for img
face 0.27 (752, 247) (777, 271)
0.08873768599960385
(416, 416, 3)
Found 1 boxes for img
face 0.73 (771, 263) (804, 303)
0.10356583599968872
(416, 416, 3)
Found 1 boxes for img
face 0.60 (787, 267) (821, 304)
0.09268284700010554
(416, 416, 3)
Found 1 boxes for img
face 0.26 (801, 276) (838, 318)
0.09466905399995085
(416, 416, 3)
Found 1 boxes for img
face 0.81 (827, 280) (863, 322)
0.09547491599960267
(416, 416, 3)
Found 1 boxes for img
face 0.92 (832, 277) (869, 323)
0.10066486699997768
(416, 416, 3)
Found 1 boxes for img
face 0.55 (857, 280) (894, 325)
0.10254184899986285
(416, 416, 3)
Found 1 boxes for img
face 0.66 (881, 289) (914, 336)
0.091784073999861
(416, 416, 

Found 0 boxes for img
0.09208650000027774
(416, 416, 3)
Found 0 boxes for img
0.0913014520001525
(416, 416, 3)
Found 0 boxes for img
0.09088929000017743
(416, 416, 3)
Found 0 boxes for img
0.08989151900004799
(416, 416, 3)
Found 0 boxes for img
0.0829533870000887
(416, 416, 3)
Found 0 boxes for img
0.09041633900005763
(416, 416, 3)
Found 0 boxes for img
0.08863574099996185
(416, 416, 3)
Found 0 boxes for img
0.09325403999991977
(416, 416, 3)
Found 0 boxes for img
0.09900217399990652
(416, 416, 3)
Found 0 boxes for img
0.08999045799964733
(416, 416, 3)
Found 0 boxes for img
0.08842581999988397
(416, 416, 3)
Found 0 boxes for img
0.0930911819996254
(416, 416, 3)
Found 0 boxes for img
0.08775967199972001
(416, 416, 3)
Found 0 boxes for img
0.09764830699987215
(416, 416, 3)
Found 0 boxes for img
0.09521910900002695
(416, 416, 3)
Found 0 boxes for img
0.0926943540002867
(416, 416, 3)
Found 0 boxes for img
0.08774822400027915
(416, 416, 3)
Found 0 boxes for img
0.09043430700012323
(416, 416,

(416, 416, 3)
Found 0 boxes for img
0.09045131999982914
(416, 416, 3)
Found 0 boxes for img
0.08999791299993376
(416, 416, 3)
Found 0 boxes for img
0.09559147600020879
(416, 416, 3)
Found 0 boxes for img
0.08787123699994481
(416, 416, 3)
Found 0 boxes for img
0.10169714699986798
(416, 416, 3)
Found 0 boxes for img
0.09765090400014742
(416, 416, 3)
Found 0 boxes for img
0.089907083000071
(416, 416, 3)
Found 0 boxes for img
0.08875729999999749
(416, 416, 3)
Found 0 boxes for img
0.09280327100032082
(416, 416, 3)
Found 0 boxes for img
0.08976955100024497
(416, 416, 3)
Found 0 boxes for img
0.0916437229998337
(416, 416, 3)
Found 0 boxes for img
0.09172962799993911
(416, 416, 3)
Found 0 boxes for img
0.09194228900014423
(416, 416, 3)
Found 0 boxes for img
0.08618574599995554
(416, 416, 3)
Found 0 boxes for img
0.08953652799982592
(416, 416, 3)
Found 0 boxes for img
0.09162274899972545
(416, 416, 3)
Found 0 boxes for img
0.10147643800019068
(416, 416, 3)
Found 0 boxes for img
0.0886304750001

(416, 416, 3)
Found 0 boxes for img
0.09090415599985135
(416, 416, 3)
Found 0 boxes for img
0.0936770510002134
(416, 416, 3)
Found 0 boxes for img
0.09094261099971845
(416, 416, 3)
Found 0 boxes for img
0.08984106200023234
(416, 416, 3)
Found 0 boxes for img
0.09455979699987438
(416, 416, 3)
Found 0 boxes for img
0.10360477700032789
(416, 416, 3)
Found 0 boxes for img
0.09441429000025892
(416, 416, 3)
Found 0 boxes for img
0.09176298500005942
(416, 416, 3)
Found 0 boxes for img
0.08994359799999074
(416, 416, 3)
Found 0 boxes for img
0.10596486799977356
(416, 416, 3)
Found 0 boxes for img
0.08882660900007977
(416, 416, 3)
Found 0 boxes for img
0.08954325700005938
(416, 416, 3)
Found 0 boxes for img
0.09221900200009259
(416, 416, 3)
Found 0 boxes for img
0.0917060759998094
(416, 416, 3)
Found 0 boxes for img
0.09237435900013224
(416, 416, 3)
Found 0 boxes for img
0.09090445899983024
(416, 416, 3)
Found 0 boxes for img
0.09517355099978886
(416, 416, 3)
Found 0 boxes for img
0.094921505000

Found 0 boxes for img
0.10055601799967917
(416, 416, 3)
Found 0 boxes for img
0.09978867700010596
(416, 416, 3)
Found 0 boxes for img
0.08397960199999943
(416, 416, 3)
Found 0 boxes for img
0.09791926599973522
(416, 416, 3)
Found 0 boxes for img
0.09721943600015948
(416, 416, 3)
Found 0 boxes for img
0.09595939200016801
(416, 416, 3)
Found 0 boxes for img
0.10190233699995588
(416, 416, 3)
Found 0 boxes for img
0.08811245900005815
(416, 416, 3)
Found 0 boxes for img
0.09418590299992502
(416, 416, 3)
Found 0 boxes for img
0.10327731000006679
(416, 416, 3)
Found 0 boxes for img
0.0987834509996901
(416, 416, 3)
Found 0 boxes for img
0.09588966399996934
(416, 416, 3)
Found 0 boxes for img
0.08738596000011967
(416, 416, 3)
Found 0 boxes for img
0.10005008100006307
(416, 416, 3)
Found 0 boxes for img
0.09318071500001679
(416, 416, 3)
Found 0 boxes for img
0.09375252499967246
(416, 416, 3)
Found 0 boxes for img
0.08977149300017118
(416, 416, 3)
Found 0 boxes for img
0.08486044700021012
(416, 4

In [ ]:
a 